In [ ]:
import numpy as np
import pickle
import random
from ikeda_V import ikeda as model
from scipy.integrate import odeint
from bootstrap_unbiased_discrete import bootstrap as PF
from generate_observations_discrete import gen_obs
from distribution_plt import distribution_plt as plt

In [ ]:
## System Parameters

# random seed
random.seed()
seed = random.getstate()

# experiment name
name ='e-10-thresh_mil_part'
name = name + '.p'
save_file = open(name,'wb')
directory = './figures_e-10-thresh_mil_part/'

# state dimension
state_dim = 2

# observation dimension
obs_dim = 2

# observation operator
H = np.eye(state_dim)

# Define the number of particles in the ensemble
particle_number = 100

# weight threshold
thresh = 1e-10

# Optional spin up step
spin = False
if spin:
    # Spin time
    spin_end = 1000

# Obs Err variance (% of climate variance) 
obs_var = 0.1

# prior covariance
p_cov = np.eye(state_dim)*.1

# Analysis performed after tanl steps
tanl = 1

# Number of Analyses (after the analysis at time zero)
nanl = 250

# Experiment length defined
exp_len = tanl*nanl

parameters = {'random_seed':seed, 'obs_var': obs_var, 'prior_cov':p_cov, 'w_threshold':thresh}

In [ ]:
## Initial conditions

# Define the initial condition for the truth
truth = np.array([.5,0])

## generate first prior and observations

# optionally spin up the trajectory for the truth
# and produce the initial prior for this state

if spin:
    spun = model(np.ones(state_dim),spin_end,1)
    spun = spun.squeeze()
    truth = spun[:,-1]

# define the prior by Gaussian with mean at the initial true state and specified covariance
P = np.eye(2)*.1
prior = np.random.multivariate_normal(truth,P,particle_number)
    
# propagate the truth for the length of the model, and return this trajector and the
# noisy observations of the state
[truth_traj,obs,Q] = gen_obs(model,truth,H,exp_len,nanl,tanl,obs_var)

In [ ]:
## particle filter step

pdf_series = PF(model,state_dim,prior,particle_number,thresh,exp_len,nanl,tanl,obs,Q)

In [ ]:
## plot and pickle data

#plot
plt(pdf_series,thresh,obs,directory)

experiment_data = {'pdf_series': pdf_series, 'observations': obs, 'parameters': parameters}

#pickle data
pickle.dump(experiment_data, save_file)
save_file.close()